# Yolo object detection
## please first download the yolo weights in section5 folder
```
wget https://pjreddie.com/media/files/yolov3.weights
```

In [3]:
import cv2
import numpy as np

classesFile = "./classes.txt"
weightsFile = "./yolov3.weights"
configFile = "./yolov3.cfg"

classes = None
COLORS = None
with open(classesFile, 'r') as f:
    classes = [line.strip() for line in f.readlines()]
    COLORS = np.random.uniform(0, 255, size=(len(classes), 3))

def get_output_layers(net):
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    return output_layers


def draw_prediction(img, class_id, confidence, x, y, x_plus_w, y_plus_h):
    label = str(classes[class_id])
    color = COLORS[class_id]
    cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 2)
    cv2.putText(img, label, (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

def find_and_show_objects(image):
    width = image.shape[1]
    height = image.shape[0]
    scale = 0.00392

    net = cv2.dnn.readNet(weightsFile, configFile)

    blob = cv2.dnn.blobFromImage(image, scale, (416,416), (0,0,0), True, crop=False)

    net.setInput(blob)

    outs = net.forward(get_output_layers(net))

    class_ids = []
    confidences = []
    boxes = []
    conf_threshold = 0.5
    nms_threshold = 0.4


    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = center_x - w / 2
                y = center_y - h / 2
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])

    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)

    for i in indices:
        i = i[0]
        box = boxes[i]
        x = box[0]
        y = box[1]
        w = box[2]
        h = box[3]
        draw_prediction(image, class_ids[i], confidences[i], round(x), round(y), round(x+w), round(y+h))
    
    return image
    
# For live webcam
cap = cv2.VideoCapture(0)

# For video record
#cap = cv2.VideoCapture('videos/damien.MOV')
while cap.isOpened():
    ret, frame = cap.read()
    if frame is None:
        break
    #frame = cv2.rotate(frame, cv2.cv2.ROTATE_90_CLOCKWISE)
    cv2.imshow('Object detection in YOLO', find_and_show_objects(frame))
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()  